In [1011]:
import pygame,sys
from pygame.locals import *
from random import randint
import math
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
import tflearn
import tensorflow as tf
from imblearn.over_sampling import SMOTE

In [1012]:
#colors
WHITE = (255, 255, 255)
BLACK=(0,0,0)

In [1013]:
def desloc(x,y,ori):
    des=1
    if(ori=='r'):
        return (x+des,y)
    elif(ori=='l'):
        return (x-des,y)
    elif(ori=='u'):
        return (x,y-des)
    elif (ori=='d'):
        return (x,y+des)

In [1014]:
def oriNumb(ori):
    
    if(ori=='r'):
        return 0
    elif(ori=='d'):
        return 1
    elif(ori=='l'):
        return 2
    elif(ori=='u'):
        return 3

def numbOri(numb):
    
    if(numb==0):
        return 'r'
    elif(numb==1):
        return 'd'
    elif(numb==2):
        return 'l'
    elif(numb==3):
        return 'u'    

In [1015]:
class food(pygame.sprite.Sprite):
    
    def __init__(self,x,y,img):
        
        pygame.sprite.Sprite.__init__(self)
        self.image=img
        self.rect=self.image.get_rect()
        self.rect.x=x
        self.rect.y=y

In [1016]:
#head
class head(pygame.sprite.Sprite):
    
    def __init__(self,x,y,img):
        
        pygame.sprite.Sprite.__init__(self)
        self.image=img
        self.rect=pygame.rect.Rect(x,y,int(img.get_width()*0.75),int(img.get_height()+0.75))
        self.orientation=''
        self.turns=[]
    
    def get_render_rect(self):
        
        return ((self.rect.x-int(self.image.get_width()*0.25)),(self.rect.y-int(self.image.get_height()*0.25)))

In [1017]:
#body
class body(pygame.sprite.Sprite):
    def __init__(self,x,y,ori,img):
        
        pygame.sprite.Sprite.__init__(self)
        self.image=img
        self.rect=pygame.rect.Rect(x,y,int(img.get_width()*0.75),int(img.get_height()*0.75))
        self.orientation=ori
        self.turns=[]
    
    def move(self):
        
        if(len(self.turns)==0):
            self.rect.x,self.rect.y=desloc(self.rect.x,self.rect.y,self.orientation)
            
        else:
            turnx,turny,turno= self.turns[0]
            virtualx,virtualy=desloc(self.rect.x,self.rect.y,self.orientation)
            
            if(((self.orientation=='r' and virtualx>turnx) or (self.orientation=='l' and virtualx<turnx) 
               or (self.orientation=='u' and virtualy<turny) or (self.orientation=='d' and virtualy>turny))
              and self.orientation!=turno):
                
                self.rect.x,self.rect.y=desloc(self.rect.x,self.rect.y,turno)
                self.orientation=turno
                self.turns.pop(0)
                
            else:
                self.rect.x,self.rect.y=desloc(self.rect.x,self.rect.y,self.orientation)

    def get_render_rect(self):
        
        return ((self.rect.x-int(self.image.get_width()*0.25)),(self.rect.y-int(self.image.get_height()*0.25)))
                

In [1018]:
#Snake
class Snake:
    
    def __init__(self,x,y,headIm,ori='r'):
        
        self.head=head(x,y,headIm)
        self.head.orientation=ori
        self.blocks=[]
        self.orientation=ori   
        self.headImg=headIm
        self.bodyImg=pygame.Surface((0,0))
        self.walls=[]
        
    def move(self,ori):
        
        if(ori!=self.orientation):
            #turn happened
            turn=(self.head.rect.x,self.head.rect.y,ori)
            for block in self.blocks:
                block.turns.append(turn)
            self.orientation=ori
            self.head.orientation=ori
   
        self.head.rect.x,self.head.rect.y=desloc(self.head.rect.x,self.head.rect.y,ori)
        
        for b in self.blocks:
            b.move()

    def createWalls(self,dim):
        
        r1=pygame.rect.Rect(0,-1,dim,1)
        r2=pygame.rect.Rect(-1,0,1,dim)
        r3=pygame.rect.Rect(dim,0,1,dim)
        r4=pygame.rect.Rect(0,dim,dim,1)
        self.walls.append(r1)
        self.walls.append(r2)
        self.walls.append(r3)
        self.walls.append(r4)
    
    def isAlive(self):
        
        head_rect=self.head.rect
        
        for bloc in self.blocks:
            if(head_rect.colliderect(bloc.rect)):
                return False
        
        for wall in self.walls:
            if(head_rect.colliderect(wall)):
                return False
        return True
    
    def calcBodyPosition(self,bloc):
        
        ori=bloc.orientation
        
        #values
        spacing=int(0.70*bloc.image.get_width())
        dist=spacing+bloc.image.get_width()
        
        #virtual values
        virtualx,virtualy=0,0
        
        if(ori=='r'):
            virtualx,virtualy=(bloc.rect.x-dist),(bloc.rect.y)

        elif(ori=='l'):
            virtualx,virtualy=(bloc.rect.x+dist),(bloc.rect.y)

        elif(ori=='u'):
            virtualx,virtualy=(bloc.rect.x),(bloc.rect.y+dist)

        elif(ori=='d'):
            virtualx,virtualy=(bloc.rect.x),(bloc.rect.y-dist)
        
        return (virtualx,virtualy)
    
    def addBlock(self):
        
        lastbloc=self.head
        
        if(len(self.blocks)>0):
            lastbloc=self.blocks[-1]
        bx,by=self.calcBodyPosition(lastbloc)
        nbody= body(bx,by,lastbloc.orientation,lastbloc.image)
        nbody.turns=list(lastbloc.turns)
        self.blocks.append(nbody)
        
    def eaten(self,food):
        
        if(self.head.rect.colliderect(food.rect)):
            return True
        return False
    
    
    def obstDist(self,ori):
        
        dx,dy=0,0
        if(ori==0):
            dx=1
        elif(ori==2):
            dx=-1
        elif(ori==3):
            dy=-1
        elif(ori==1):
            dy=1
        
        virtualx,virtualy=int(self.head.rect.x+(self.head.image.get_width()*0.5)),int(self.head.rect.y+(self.head.image.get_height()*0.5))
        flag=True
        while(flag):
            virtualx+=dx
            virtualy+=dy
            for bloc in self.blocks:
                bx,by=bloc.get_render_rect()
                sp=int(bloc.image.get_width()*0.7)
                nrect=pygame.rect.Rect(bx,by,(bloc.image.get_width()+sp),(bloc.image.get_height()+sp))
                if(nrect.collidepoint(virtualx,virtualy)):
                    flag=False
                    break
            
            if(flag):    
                for wall in self.walls:
                    if(wall.collidepoint(virtualx,virtualy)):
                        flag=False
                        break
        if(ori==0 or ori==2):
            return abs((self.head.rect.x-virtualx))
        elif(ori==1 or ori==3):
            return abs((self.head.rect.y-virtualy))

    
    def getAtr(self):
        
        nori=oriNumb(self.orientation)
        return [self.obstDist((nori+3)%4),self.obstDist(nori),self.obstDist((nori+1)%4)]

In [1019]:
def renderSnake(display,snake):
    
    headImg=snake.headImg
    bodyImg=snake.bodyImg
    
    #add head
    display.blit(headImg,snake.head.get_render_rect())
    
    #add body
    for body in snake.blocks:
        display.blit(bodyImg,body.get_render_rect()) 
        

In [1020]:
def renderFood(display,food):
    
    display.blit(food.image,food.rect)

In [1021]:
def newFood(foodImg,dim):
    
    fos=5
    return food(randint(fos,(dim-foodImg.get_width()-fos)),randint(fos,(dim-foodImg.get_height()-fos)),foodImg)

In [1022]:
def foodPos(snake,food):
    
    hip=math.sqrt((snake.head.rect.x-food.rect.x)**2+(snake.head.rect.y-food.rect.y)**2)
    cat=0
    if(snake.orientation=='r'):
        cat=food.rect.x-snake.head.rect.x
        if(food.rect.y>=snake.head.rect.y):
            return (360-math.degrees(np.arccos(cat/hip))),hip
        else:
            return math.degrees(np.arccos(cat/hip)),hip

    elif(snake.orientation=='l'):
        cat=snake.head.rect.x-food.rect.x
        if(food.rect.y<=snake.head.rect.y):
            return (360-math.degrees(np.arccos(cat/hip))),hip
        else:
            return math.degrees(np.arccos(cat/hip)),hip
    
    elif(snake.orientation=='u'):
        cat=snake.head.rect.y-food.rect.y
        if(food.rect.x<=snake.head.rect.x):
            return (360-math.degrees(np.arccos(cat/hip))),hip
        else:
            return math.degrees(np.arccos(cat/hip)),hip
    
    elif(snake.orientation=='d'):
        cat=food.rect.y-snake.head.rect.y
        if(food.rect.x>=snake.head.rect.x):
            return (360-math.degrees(np.arccos(cat/hip))),hip
        else:
            return math.degrees(np.arccos(cat/hip)),hip
     

In [1023]:
#reset files

#pickle.dump( [], open( "features.fer", "wb" ) )    
#pickle.dump( [], open( "tags.fer", "wb" ) )


In [1024]:
def calcTag(lastori,ori):
    
    resp=oriNumb(ori)-oriNumb(lastori)
    if((lastori=='u' and ori=='r')):
        resp=1
    elif((lastori=='r' and ori=='u')):
        resp=-1
    return resp

In [1025]:
def Create_Classifier(name):
	if(name=='LogReg'):
		return linear_model.LogisticRegression(n_jobs=1, C=1e5)
	elif(name=='KNN'):
		return KNeighborsClassifier(n_neighbors=3, n_jobs=1, algorithm='brute', metric='cosine')
	elif(name=='NB'):
		return MultinomialNB(alpha=0.01)
	elif(name=='MLPerceptron'):
		return MLPClassifier()
	else:
		raise NameError('Classifier Unavailable')

In [ ]:
#Tensor Flow 

def fully_connected_layer(input, n_inp, n_out, use_relu=True )

In [1026]:
#init
print("Initializing game")
pygame.init()
dim=600
displaysurf=pygame.display.set_mode((dim,dim))
pygame.display.set_caption('Snaike')
fpsClock = pygame.time.Clock()
FPS = 120
framecount=0


#emilia variables
emiliaImg = pygame.image.load('blue.png')
emiliaSnake = Snake(int(dim/2)+100,int(dim/2)+100,emiliaImg,ori='u')
emiliaSnake.createWalls(dim)

#subaro
subaroImg=pygame.image.load('blue.png')
emiliaSnake.bodyImg=subaroImg

#game over
gameoverImg=pygame.image.load('gameover.jpg')

#food
foodImg=pygame.image.load('hamb.png')
foood=newFood(foodImg,dim)

#orientation
lastorientation=emiliaSnake.orientation
orientation=emiliaSnake.orientation
gameOverFlag=False

print("Opening config files")
#training features and tags
features=pickle.load( open( "features.fer", "rb" ) )
tags=pickle.load( open("tags.fer","rb"))
print("Features: ", len(features))

print("Test-split")
#Training and test features
X_train, X_test, y_train, y_test = train_test_split( features, tags, test_size=0.10, random_state=42)

print("Smoting")
#Smote
sm = SMOTE(random_state=42)
X_train,y_train=sm.fit_sample(X_train,y_train)
print("Features: ", len(X_train))

print("Training model ...")
#Neural net
aux=features[0]
x=tf.placeholder(tf.float32,shape=[None,len(aux)], name='x')


print(classification_report(y_test,pred))

print("game starts")

print(features, tags)
#game loop
while True:   
    displaysurf.fill(WHITE)
    
    for event in pygame.event.get():
        if event.type== QUIT:
            #pickle.dump( features, open( "features.fer", "wb" ) )    
            #pickle.dump( tags, open( "tags.fer", "wb" ) )
            pygame.quit()
            sys.exit()
        elif event.type==KEYDOWN:
            if event.key==K_UP and orientation!='d':
                orientation='u'
            elif event.key==K_DOWN and orientation!='u':
                orientation='d'
            elif event.key==K_RIGHT and orientation!='l':
                orientation='r'
            elif event.key==K_LEFT and orientation!='r':
                orientation='l'
            elif event.key==K_f:
                emiliaSnake = Snake(int(dim/2),int(dim/2),emiliaImg)
                emiliaSnake.createWalls(dim)
                emiliaSnake.bodyImg=subaroImg
                gameOverFlag=False
            elif event.key==K_n:
                #pickle.dump( features, open( "features.fer", "wb" ) )
                #pickle.dump( tags, open( "tags.fer", "wb" ) )
                pygame.quit()
                sys.exit()
    
    framecount=(framecount+1)%30
    
    atr1,atr2,atr3,ang,hip=0,0,0,0,0
    #emilia motion
    if(emiliaSnake.isAlive()):
        
        if(framecount==0):
            atr1,atr2,atr3=emiliaSnake.getAtr()
            ang,hip= foodPos(emiliaSnake,foood)

            mv=model.predict([[atr1,atr2,atr3,ang,hip]])
            print(mv)
            if(orientation=='u' and mv==1):
                orientation='r'
            elif(orientation=='r' and mv==-1):
                orientation='u'
            else:
                orientation=numbOri(mv+oriNumb(orientation))
        
        emiliaSnake.move(orientation)
    
    if(not emiliaSnake.isAlive()):
        gameOverFlag=True
        
        displaysurf.blit(gameoverImg, (int((dim-gameoverImg.get_width())/2),(int((dim-gameoverImg.get_height())/2)-50)))
        pygame.display.update()

    if(emiliaSnake.eaten(foood)):
        
        foood=newFood(foodImg,dim)
        emiliaSnake.addBlock()
    
    if(not gameOverFlag):
        
        if(framecount==0 or lastorientation!=orientation):
            
            atr1,atr2,atr3=emiliaSnake.getAtr()
            ang,hip= foodPos(emiliaSnake,foood)
            features.append([atr1,atr2,atr3,ang])
            rio=calcTag(lastorientation,orientation)
            tags.append(rio)
            lastorientation=orientation
            
        renderSnake(displaysurf,emiliaSnake)
        renderFood(displaysurf,foood)
        pygame.display.update()
        
    fpsClock.tick(FPS)
    

            

Initializing game
Opening config files
Features:  2160
Test-split
Smoting
Features:  4989
Training model ...
             precision    recall  f1-score   support

         -1       0.45      0.93      0.60        14
          0       0.99      0.84      0.91       180
          1       0.55      0.82      0.65        22

avg / total       0.91      0.85      0.86       216

game starts
[[401, 371, 200, 330.02400386756625, 338.2454729926182], [401, 341, 200, 327.27578579377894, 312.61797773000836], [401, 311, 200, 324.0458012615908, 287.8367592924851], [401, 281, 200, 320.22220194458674, 264.14011433328335], [401, 251, 200, 315.6700955222245, 241.84705910967782], [401, 221, 200, 310.23635830927384, 221.38202275704322], [401, 191, 200, 303.76823361597553, 203.29781110479277], [401, 161, 200, 296.15677462305297, 188.2817038376273], [401, 131, 200, 287.41191863425416, 177.11578134090706], [401, 101, 200, 277.75003642291927, 170.55790805471318], [515, 400, 86, 357.27368900609366, 168.190368

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]


SystemExit: 

C:\Users\Fernando\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
